In [2]:
from Bio import SeqIO, Entrez
import os
from urllib.error import HTTPError
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from itertools import permutations, product
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
import tqdm
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, cross_val_score

from numpy import mean
from numpy import std
import pickle
from os import path
from sklearn.model_selection import cross_val_score
from warnings import simplefilter
from collections import OrderedDict
from sklearn.metrics import accuracy_score, auc, confusion_matrix, balanced_accuracy_score, precision_recall_curve, auc, roc_curve, roc_auc_score

from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)

Entrez.tool = "Zoonosis predictor"

Entrez.email = input("Enter an email address to use NCBI e-utils: ")

In [31]:

def resetkmerdict(permset)->OrderedDict:
        kmerdict = OrderedDict()
        for i in permset:
            kmerdict[i]=0
        return kmerdict

def assign_kmers_to_dict(row, permset, kmer):
    kmerdict=resetkmerdict(permset)
    st = row # tune for which column the sequence is in
    for j in range(len(st)-kmer+1):
        if not st[j:j+kmer] in permset: continue
        kmerdict[st[j:j+kmer]]+=1
    return kmerdict

def getTrainParams(mergedDf, kmer):
    print(mergedDf)
    s = product('acgt',repeat = kmer)
    permset = set(["".join(x) for x in list(s)])

    l = []
    
    for row in tqdm.tqdm(mergedDf.itertuples()):
        l.append(assign_kmers_to_dict(row, permset, kmer))

    finalkmerdict=pd.DataFrame(l)
    
    # shouldn't need to fill NAs
    # mergedDf.fillna(0, inplace=True)

    X = finalkmerdict
    Y = mergedDf['isZoonotic']
    X = X.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=1)
    # print(X.head())

    global asdfX
    asdfX = X.copy()
    global asdfY
    asdfY = Y.copy()


    place = pd.concat([X, Y], axis=1)
    
    # print(place)

    place.to_csv('data/info.csv', index=False)

    return train_test_split(X, Y, test_size=0.2, random_state=1)

In [33]:
best_gradBoost = pickle.load(open('models/curr_models/gradBoost.pkl', 'rb'))
best_xgBoost = pickle.load(open('models/curr_models/xgBoost.pkl', 'rb'))
nardus_model = pickle.load(open('models/nardus_testing/gradBoost.pkl', 'rb'))
gridsearch = pickle.load(open('models/gridsearch/gradBoost.pkl', 'rb')) # poorest model


kmer = 4

s = product('acgt',repeat = kmer)
permset = set(["".join(x) for x in list(s)])

preds = []
for ind, file in enumerate(os.listdir("data/virome_contigs")):
    # print(ind)
    fasta_sequences = SeqIO.parse(open(f"data/virome_contigs/{file}"),'fasta')

    fastas = [x for x in fasta_sequences]
    # print(fasta)
    for fasta in fastas:
        name, sequence = fasta.id, str(fasta.seq)
        X_info = sequence.lower()
        # print(X_info[:10])

        oDict = assign_kmers_to_dict(X_info, permset, kmer) # convert ordereddict to pandas dataframe

        kmer_df = pd.DataFrame()

        for i in oDict:
            kmer_df.at[0, i]=oDict[i]
        

        cols_when_model_builds = best_gradBoost.feature_names_in_
        kmer_df=kmer_df[cols_when_model_builds]
        
        gradboost = best_gradBoost.predict_proba(kmer_df)[:,1]
        
        cols_when_model_builds = best_xgBoost.get_booster().feature_names
        kmer_df=kmer_df[cols_when_model_builds]

        xgpred = best_xgBoost.predict_proba(kmer_df)[:,1]

        cols_when_model_builds = nardus_model.feature_names_in_
        kmer_df=kmer_df[cols_when_model_builds]
        
        narduspred = nardus_model.predict_proba(kmer_df)[:,1]

        cols_when_model_builds = gridsearch.feature_names_in_
        kmer_df=kmer_df[cols_when_model_builds]
        
        gridsearchpred = gridsearch.predict_proba(kmer_df)[:,1]

        preds.append([gradboost, xgpred, narduspred, gridsearchpred])

preds = np.array(preds)



    # print(best_gradBoost.predict(kmer_df), sequences.loc[ind]['isZoonotic'])
    # print(best_gradBoost.predict(kmer_df), sequences['isZoonotic'])
        # print(accuracy_score())

In [35]:
print(preds[:50])

[[[0.99472879]
  [0.17953709]
  [0.99991616]
  [0.98351803]]

 [[0.99472879]
  [0.17953709]
  [0.99991616]
  [0.98351803]]

 [[0.99472879]
  [0.17953709]
  [0.99991616]
  [0.98351803]]

 [[0.81636603]
  [0.86654699]
  [0.99997701]
  [0.58062318]]

 [[0.89563144]
  [0.62301123]
  [0.99994731]
  [0.41822736]]

 [[0.9882871 ]
  [0.15187934]
  [0.99999928]
  [0.91920053]]

 [[0.84290229]
  [0.65225333]
  [0.99999995]
  [0.99952905]]

 [[0.99974911]
  [0.52138221]
  [0.99979949]
  [0.94473588]]

 [[0.86454886]
  [0.31796557]
  [0.99973965]
  [0.98199736]]

 [[0.94852308]
  [0.38062403]
  [0.9998544 ]
  [0.99954155]]

 [[0.99530284]
  [0.53400129]
  [0.99999976]
  [0.99022732]]

 [[0.98089831]
  [0.47641757]
  [0.99933549]
  [0.97996587]]

 [[0.99932044]
  [0.23872426]
  [0.99987933]
  [0.96729291]]

 [[0.4596301 ]
  [0.54573119]
  [0.99991765]
  [0.97438115]]

 [[0.99825051]
  [0.67055488]
  [0.99943961]
  [0.92945402]]

 [[0.99994481]
  [0.45912325]
  [0.82781363]
  [0.96775836]]

 [[0.989

In [ ]:
print(sequence['isZoonotic'])